In [ ]:
import requests

# Banned Player List 가져오기
url = "https://api.gametools.network/"
response = requests.get(url+"bfban/banned_players/")

banned_user_ids = []
if response.status_code == 200:
    # 성공적으로 데이터를 받아옴
    data = response.json() # JSON 응답을 파이썬 딕셔너리로 변환
    banned_user_ids = data["personaids"]
else:
    # 오류 처리
    print(f'Error: {response.status_code}')

In [ ]:
# Banned Player List 확인
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import pandas as pd
from tqdm import tqdm
import time

# 데이터를 저장할 리스트 초기화
data = []

# Chrome 바이너리 위치와 ChromeDriver 경로 설정
chrome_binary_path = "E:/Github/BattleField-Bot-Detection/dataset/src/chrome/chrome.exe"  # Chrome 실행 파일 위치
chromedriver_path = "E:/Github/BattleField-Bot-Detection/dataset/src/chrome/chromedriver.exe"  # ChromeDriver 경로

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.binary_location = chrome_binary_path  # Chrome 바이너리 위치 지정
driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)

# 각 사용자 ID에 대해 반복
for idx, user_id in tqdm(enumerate(banned_user_ids)):
    url = f'https://bfban.com/player/{user_id}'
    driver.get(url)
    
    # 페이지 로딩 대기
    if idx % 2 == 0:
        time.sleep(10)
    elif idx % 10 == 0:
        time.sleep(30)
    else:
        time.sleep(5)
    
    try:
        user_type = driver.find_element(By.CLASS_NAME, "ivu-tag-text.ivu-tag-color-white").text if driver.find_elements(By.CLASS_NAME, "ivu-tag-text.ivu-tag-color-white") else ''
        
        game_img_src = driver.find_element(By.CSS_SELECTOR, ".ivu-tag-text.ivu-tag-color-default img").get_attribute('src') if driver.find_elements(By.CSS_SELECTOR, ".ivu-tag-text.ivu-tag-color-default img") else ''
        
        # 여러 <span> 요소들 중 최대 3개까지 추출
        bot_types = driver.find_elements(By.CSS_SELECTOR, ".ivu-tag.ivu-tag-size-default.ivu-tag-warning.ivu-tag-checked > span")
        bot_type_texts = [element.text for element in bot_types[:3]]  # 최대 3개의 요소 텍스트 추출
        
        date_times = []
        datetime_element = driver.find_elements(By.CSS_SELECTOR, "span.ivu-time")
        for element in datetime_element:
            if element.text == "":
                continue
            elif element.text != "":
                date_times.append(element.text)
            else:
                pass
        
    except Exception as e:
        print(f"Error retrieving data for user ID {user_id}: {e}")
        user_type = game_img_src = ''
        bot_type_texts = []
        date_times = []
    
    # 추출된 텍스트들을 데이터에 추가
    data_entry = {
        'Type': user_type,
        'ID': user_id,
        'game_img_src': game_img_src,
        'Firstly reported date': date_times[1] if len(date_times) > 0 else '',
        'Recently reported date': date_times[2] if len(date_times) > 0 else ''
    }
    
    # 동적으로 키 생성하여 추출된 각 bot_type 저장
    for i, bot_type_text in enumerate(bot_type_texts, start=1):
        data_entry[f'bot_type_{i}'] = bot_type_text
    
    data.append(data_entry)
    print(data)
# 드라이버 종료
driver.quit()

# 데이터프레임 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
csv_file_path = './band_user_data_selenium.csv'
df.to_csv(csv_file_path, index=False)

print(f'CSV 파일이 생성되었습니다: {csv_file_path}')
